In [1]:
import sys
sys.path.append("..")
import numpy as np
import focusadd
from focusadd.surface.Surface import Surface
from focusadd.coils.CoilSet import CoilSet
import numpy as np
import mayavi as maya
from mayavi import mlab
mlab.init_notebook()

Notebook initialized with x3d backend.


# Validate CoilSet.Py 

# Test 1: Coil Initialization

Let's test the coil initialization. There are two ways of initializing coils, either from a preformatted args_dict dictionary or from an existing .hdf5 file which stores the coil data and metadata. Let's make sure we can do this successfully. 

Let's start by looking at a basic circular axis and circular cross section, and making sure we can properly initialize the coils around this axis. Then we'll make sure we can write these to an .hdf5 file, then read them in again.

In [2]:
mlab.clf()
surface = Surface("../focusadd/initFiles/axes/circularAxis.txt", 64, 32, 1.0)
r = surface.get_r()
x = r[:,:,0]
y = r[:,:,1]
z = r[:,:,2]
s = mlab.mesh(x,y,z,color=(0.8,0.0,0.0))
args_dict = {}
args_dict['numCoils'] = 8
args_dict['numSegments'] = 32
args_dict['numFourierCoils'] = 4
args_dict['numFourierRotate'] = 4
args_dict['lengthNormal'] = 0.02
args_dict['lengthBinormal'] = 0.02
args_dict['numNormalRotate'] = 2
args_dict['numBinormalRotate'] = 2
args_dict['radiusCoil'] = 2.0
args_dict['numRotate'] = 0
coilSet = CoilSet(surface,args_dict = args_dict)

/opt/anaconda3/lib/python3.7/site-packages/jax/lib/xla_bridge.py:119: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [3]:
r_coils = coilSet.get_r()
for ic in range(r_coils.shape[0]):
    for n in range(r_coils.shape[2]):
        for b in range(r_coils.shape[3]):
            p = mlab.plot3d(r_coils[ic,:,n,b,0],r_coils[ic,:,n,b,1],r_coils[ic,:,n,b,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.0,0.8))

In [4]:
p

It should work for any number of coils. 

In [ ]:
mlab.clf()
p = mlab.mesh(x,y,z,color=(0.8,0.0,0.0))
args_dict['numCoils'] = 13
coilSet = CoilSet(surface,args_dict = args_dict)
r_coils = coilSet.get_r()
for ic in range(r_coils.shape[0]):
    for n in range(r_coils.shape[2]):
        for b in range(r_coils.shape[3]):
            p = mlab.plot3d(r_coils[ic,:,n,b,0],r_coils[ic,:,n,b,1],r_coils[ic,:,n,b,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.0,0.8))

In [ ]:
p

13 coils, evenly spaced. Great. 

The next step is to store this as an hdf5 file, then read it back in and get the same coils. Let's do that and make sure it works. 

In [ ]:
coilSet.write("13coilsTest.hdf5")

In [ ]:
coilSet13 = CoilSet(surface,input_file="13coilsTest.hdf5")

In [ ]:
mlab.clf()
p = mlab.mesh(x,y,z,color=(0.8,0.0,0.0))
r_coils = coilSet13.get_r()
for ic in range(r_coils.shape[0]):
    for n in range(r_coils.shape[2]):
        for b in range(r_coils.shape[3]):
            p = mlab.plot3d(r_coils[ic,:,n,b,0],r_coils[ic,:,n,b,1],r_coils[ic,:,n,b,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.0,0.8))

In [ ]:
p

coilSet13 looks good. So writing and reading from hdf5 files gives us the same coils. 

Now let's see if we properly initialize our coils around an elliptical axis. 

In [ ]:
mlab.clf()
surface = Surface("../focusadd/initFiles/axes/ellipticalAxis5Rotate.txt", 64, 32, 1.0)
r = surface.get_r()
x = r[:,:,0]
y = r[:,:,1]
z = r[:,:,2]
p = mlab.mesh(x,y,z,color=(0.8,0.0,0.0))
args_dict = {}
args_dict['numCoils'] = 8
args_dict['numSegments'] = 32
args_dict['numFourierCoils'] = 4
args_dict['numFourierRotate'] = 4
args_dict['lengthNormal'] = 0.02
args_dict['lengthBinormal'] = 0.02
args_dict['numNormalRotate'] = 2
args_dict['numBinormalRotate'] = 2
args_dict['radiusCoil'] = 2.0
args_dict['numRotate'] = 0
coilSet = CoilSet(surface,args_dict = args_dict)

In [ ]:
r_coils = coilSet.get_r()
for ic in range(r_coils.shape[0]):
    for n in range(r_coils.shape[2]):
        for b in range(r_coils.shape[3]):
            p = mlab.plot3d(r_coils[ic,:,n,b,0],r_coils[ic,:,n,b,1],r_coils[ic,:,n,b,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.0,0.8))

In [ ]:
p

# Test 2: Frenet Frame

Let's look at the tangent, normal, and binormal vectors for coils initialized around an elliptical surface. Hopefully they match our expectations, a more thorough debugging might be challenging. 

For ease of viewing, I only plotted the central coil filament.

In [ ]:
surface = Surface("../focusadd/initFiles/axes/ellipticalAxis5Rotate.txt", 64, 32, 1.0)
r = surface.get_r()
x = r[:,:,0]
y = r[:,:,1]
z = r[:,:,2]
args_dict = {}
args_dict['numCoils'] = 8
args_dict['numSegments'] = 32
args_dict['numFourierCoils'] = 4
args_dict['numFourierRotate'] = 4
args_dict['lengthNormal'] = 0.02
args_dict['lengthBinormal'] = 0.02
args_dict['numNormalRotate'] = 2
args_dict['numBinormalRotate'] = 2
args_dict['radiusCoil'] = 2.0
args_dict['numRotate'] = 0
coilSet = CoilSet(surface,args_dict = args_dict)

In [ ]:
mlab.clf()
r_coils = coilSet.get_r()
r_central = coilSet.get_r_central()
T = coilSet.get_tangent()
N = coilSet.get_normal()
B = coilSet.get_binormal()
p = mlab.mesh(x,y,z,color=(0.8,0.0,0.0))
for ic in range(r_coils.shape[0]):
    p = mlab.quiver3d(r_central[::,::,0],r_central[::,::,1],r_central[::,::,2],T[::,::,0],T[::,::,1],T[::,::,2])
    p = mlab.plot3d(r_central[ic,:,0],r_central[ic,:,1],r_central[ic,:,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.0,0.8))

In [ ]:
p

In [ ]:
mlab.clf()
r_coils = coilSet.get_r()
r_central = coilSet.get_r_central()
T = coilSet.get_tangent()
N = coilSet.get_normal()
B = coilSet.get_binormal()
for ic in range(r_coils.shape[0]):
    p = mlab.quiver3d(r_central[::,::,0],r_central[::,::,1],r_central[::,::,2],N[::,::,0],N[::,::,1],N[::,::,2],color=(0.8,0.0,0.0))
    p = mlab.plot3d(r_central[ic,:,0],r_central[ic,:,1],r_central[ic,:,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.0,0.8))

In [ ]:
p

In [ ]:
mlab.clf()
r_coils = coilSet.get_r()
r_central = coilSet.get_r_central()
T = coilSet.get_tangent()
N = coilSet.get_normal()
B = coilSet.get_binormal()
#p = mlab.mesh(x,y,z,color=(0.8,0.0,0.0))
for ic in range(r_coils.shape[0]):
    p = mlab.quiver3d(r_central[::,::,0],r_central[::,::,1],r_central[::,::,2],B[::,::,0],B[::,::,1],B[::,::,2],color=(0.8,0.0,0.0))
    p = mlab.plot3d(r_central[ic,:,0],r_central[ic,:,1],r_central[ic,:,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.0,0.8))

In [ ]:
p

Seems right. 

# Test 3: Multi-filament parametrization 

Let's make sure our winding pack approximation to the finite-build coils is doing what we want it to be doing. Let's start by varying arg_dict and seeing how that changes the properties of our multi-filament coils. 

Let's vary: 

1. numFourierCoils (NF)
2. numFourierRotate (NFR)
3. lengthNormal (ln or l1)
4. lengthBinormal (lb or l2)
5. radiusCoil 
6. numNormalRotate (NNR)
7. numBinormalRotate (NBR)
8. numRotate
9. Fourier series of rotation

Here are our default settings. 

In [5]:
mlab.clf()
surface = Surface("../focusadd/initFiles/axes/ellipticalAxis5Rotate.txt", 64, 32, 1.0)
r = surface.get_r()
x = r[:,:,0]
y = r[:,:,1]
z = r[:,:,2]
#p = mlab.mesh(x,y,z,color=(0.8,0.0,0.0))
args_dict = {}
args_dict['numCoils'] = 8
args_dict['numSegments'] = 32
args_dict['numFourierCoils'] = 4
args_dict['numFourierRotate'] = 4
args_dict['lengthNormal'] = 0.02
args_dict['lengthBinormal'] = 0.02
args_dict['numNormalRotate'] = 2
args_dict['numBinormalRotate'] = 2
args_dict['radiusCoil'] = 2.0
args_dict['numRotate'] = 0
coilSet = CoilSet(surface,args_dict = args_dict)
r_coils = coilSet.get_r()
for ic in range(1):
    for n in range(r_coils.shape[2]):
        for b in range(r_coils.shape[3]):
            p = mlab.plot3d(r_coils[ic,:,n,b,0],r_coils[ic,:,n,b,1],r_coils[ic,:,n,b,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.0,0.8))
p

1. numFourierCoils (NF)

In [ ]:
mlab.clf()
args_dict['numFourierCoils'] = 2
coilSet = CoilSet(surface,args_dict = args_dict)
r_coils = coilSet.get_r()
for ic in range(1):
    for n in range(r_coils.shape[2]):
        for b in range(r_coils.shape[3]):
            p = mlab.plot3d(r_coils[ic,:,n,b,0],r_coils[ic,:,n,b,1],r_coils[ic,:,n,b,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.0,0.8))
p

In [ ]:
mlab.clf()
args_dict['numFourierCoils'] = 3
coilSet = CoilSet(surface,args_dict = args_dict)
r_coils = coilSet.get_r()
for ic in range(1):
    for n in range(r_coils.shape[2]):
        for b in range(r_coils.shape[3]):
            p = mlab.plot3d(r_coils[ic,:,n,b,0],r_coils[ic,:,n,b,1],r_coils[ic,:,n,b,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.0,0.8))
p

In [ ]:
args_dict['numFourierCoils'] = 4

2. numFourierRotate (NFR)

In [ ]:
mlab.clf()
args_dict['numFourierRotate'] = 1
coilSet = CoilSet(surface,args_dict = args_dict)
r_coils = coilSet.get_r()
for ic in range(1):
    for n in range(r_coils.shape[2]):
        for b in range(r_coils.shape[3]):
            p = mlab.plot3d(r_coils[ic,:,n,b,0],r_coils[ic,:,n,b,1],r_coils[ic,:,n,b,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.0,0.8))
p

In [ ]:
mlab.clf()
args_dict['numFourierRotate'] = 0
coilSet = CoilSet(surface,args_dict = args_dict)
r_coils = coilSet.get_r()
for ic in range(1):
    for n in range(r_coils.shape[2]):
        for b in range(r_coils.shape[3]):
            p = mlab.plot3d(r_coils[ic,:,n,b,0],r_coils[ic,:,n,b,1],r_coils[ic,:,n,b,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.0,0.8))
p

In [ ]:
args_dict['numFourierRotate'] = 4

3. lengthNormal (ln or l1)

In [ ]:
mlab.clf()
args_dict['lengthNormal'] = 0.04
coilSet = CoilSet(surface,args_dict = args_dict)
r_coils = coilSet.get_r()
for ic in range(1):
    for n in range(r_coils.shape[2]):
        for b in range(r_coils.shape[3]):
            p = mlab.plot3d(r_coils[ic,:,n,b,0],r_coils[ic,:,n,b,1],r_coils[ic,:,n,b,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.0,0.8))
p

In [ ]:
mlab.clf()
args_dict['lengthNormal'] = 0.08
coilSet = CoilSet(surface,args_dict = args_dict)
r_coils = coilSet.get_r()
for ic in range(1):
    for n in range(r_coils.shape[2]):
        for b in range(r_coils.shape[3]):
            p = mlab.plot3d(r_coils[ic,:,n,b,0],r_coils[ic,:,n,b,1],r_coils[ic,:,n,b,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.0,0.8))
p

In [ ]:
args_dict['lengthNormal'] = 0.02

4. lengthBinormal (lb or l2)

In [ ]:
mlab.clf()
args_dict['lengthBinormal'] = 0.08
coilSet = CoilSet(surface,args_dict = args_dict)
r_coils = coilSet.get_r()
for ic in range(1):
    for n in range(r_coils.shape[2]):
        for b in range(r_coils.shape[3]):
            p = mlab.plot3d(r_coils[ic,:,n,b,0],r_coils[ic,:,n,b,1],r_coils[ic,:,n,b,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.0,0.8))
p

In [ ]:
args_dict['lengthBinormal'] = 0.02

5. radiusCoil 

In [ ]:
mlab.clf()
args_dict['radiusCoil'] = 4.0
coilSet = CoilSet(surface,args_dict = args_dict)
r_coils = coilSet.get_r()
for ic in range(1):
    for n in range(r_coils.shape[2]):
        for b in range(r_coils.shape[3]):
            p = mlab.plot3d(r_coils[ic,:,n,b,0],r_coils[ic,:,n,b,1],r_coils[ic,:,n,b,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.0,0.8))
p

In [ ]:
mlab.clf()
args_dict['radiusCoil'] = 8.0
coilSet = CoilSet(surface,args_dict = args_dict)
r_coils = coilSet.get_r()
for ic in range(1):
    for n in range(r_coils.shape[2]):
        for b in range(r_coils.shape[3]):
            p = mlab.plot3d(r_coils[ic,:,n,b,0],r_coils[ic,:,n,b,1],r_coils[ic,:,n,b,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.0,0.8))
p

In [ ]:
args_dict['radiusCoil'] = 2.0

6. numNormalRotate (NNR)

In [ ]:
mlab.clf()
args_dict['numNormalRotate'] = 4
coilSet = CoilSet(surface,args_dict = args_dict)
r_coils = coilSet.get_r()
for ic in range(1):
    for n in range(r_coils.shape[2]):
        for b in range(r_coils.shape[3]):
            p = mlab.plot3d(r_coils[ic,:,n,b,0],r_coils[ic,:,n,b,1],r_coils[ic,:,n,b,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.0,0.8))
p

In [ ]:
args_dict['numNormalRotate'] = 2

7. numBinormalRotate (NBR)

In [ ]:
mlab.clf()
args_dict['numBinormalRotate'] = 4
coilSet = CoilSet(surface,args_dict = args_dict)
r_coils = coilSet.get_r()
for ic in range(1):
    for n in range(r_coils.shape[2]):
        for b in range(r_coils.shape[3]):
            p = mlab.plot3d(r_coils[ic,:,n,b,0],r_coils[ic,:,n,b,1],r_coils[ic,:,n,b,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.0,0.8))
p

In [ ]:
args_dict['numBinormalRotate'] = 2

8. numRotate

In [ ]:
mlab.clf()
args_dict['numRotate'] = 1
coilSet = CoilSet(surface,args_dict = args_dict)
r_coils = coilSet.get_r()
for ic in range(1):
    for n in range(r_coils.shape[2]):
        for b in range(r_coils.shape[3]):
            p = mlab.plot3d(r_coils[ic,:,n,b,0],r_coils[ic,:,n,b,1],r_coils[ic,:,n,b,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.0,0.8))
p

In [ ]:
mlab.clf()
args_dict['numRotate'] = 2
coilSet = CoilSet(surface,args_dict = args_dict)
r_coils = coilSet.get_r()
for ic in range(1):
    for n in range(r_coils.shape[2]):
        for b in range(r_coils.shape[3]):
            p = mlab.plot3d(r_coils[ic,:,n,b,0],r_coils[ic,:,n,b,1],r_coils[ic,:,n,b,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.0,0.8))
p

In [ ]:
mlab.clf()
args_dict['numRotate'] = 3
coilSet = CoilSet(surface,args_dict = args_dict)
r_coils = coilSet.get_r()
for ic in range(1):
    for n in range(r_coils.shape[2]):
        for b in range(r_coils.shape[3]):
            p = mlab.plot3d(r_coils[ic,:,n,b,0],r_coils[ic,:,n,b,1],r_coils[ic,:,n,b,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.0,0.8))
p

In [ ]:
args_dict['numRotate'] = 0

9. Fourier series of rotation

In [13]:
fc, fr = coilSet.get_params()

In [14]:
print(fr.shape)

(2, 8, 4)


In [19]:
from jax.ops import index, index_update
fr = index_update(fr, index[0,0,0],3.14/4.)
coilSet.set_params((fc,fr))

In [20]:
mlab.clf()
r_coils = coilSet.get_r()
for ic in range(1):
    for n in range(r_coils.shape[2]):
        for b in range(r_coils.shape[3]):
            p = mlab.plot3d(r_coils[ic,:,n,b,0],r_coils[ic,:,n,b,1],r_coils[ic,:,n,b,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.0,0.8))
p

In [21]:
fr = index_update(fr, index[0,0,0],0.0)
fr = index_update(fr, index[0,0,1],3.14/2)
coilSet.set_params((fc,fr))

In [22]:
mlab.clf()
r_coils = coilSet.get_r()
for ic in range(1):
    for n in range(r_coils.shape[2]):
        for b in range(r_coils.shape[3]):
            p = mlab.plot3d(r_coils[ic,:,n,b,0],r_coils[ic,:,n,b,1],r_coils[ic,:,n,b,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.0,0.8))
p

Looks correct. Good. 